In [ ]:
import datasets
import os
import openai
import numpy as np
with open(os.path.expanduser('~/.openai_api_key'), 'r') as file:
    openai.api_key = file.read().replace('\n', '')

import adatest
import re

/gscratch/cse/bparan/tools/envs/bigben/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-04 08:47:03.399700: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-04 08:47:03.404284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-04 08:47:03.404303: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
d = datasets.load_dataset('bigbench', 'anachronisms')

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
import numpy as np
inputs = d['train']['inputs'] + d['validation']['inputs']
inputs = [x.split('\n')[0] for x in inputs]
labels = np.array([int(x[0] == 'Yes') for x in d['train']['targets'] + d['validation']['targets']])

In [5]:
class OpenAIModel(adatest.Model):
    def __init__(self, model="text-davinci-002", quote="\"", temperature=0.7, top_p=1, max_length=30, n=1):
        self.model = model
        self.api_key = openai.api_key
        self.quote = quote
        self.temperature = temperature
        self.top_p = top_p
        self.max_length = max_length
        self.n = n
    def __call__(self, strings):
        resp = openai.Completion.create(
            model=self.model,
            prompt=strings,
            max_tokens=self.max_length,
            temperature=self.temperature,
            top_p=self.top_p,
            n=self.n,
            stop=self.quote,
        )
        return [x["text"] for x in resp['choices']]

gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='', n=1)


In [6]:
def propose_decomposition(n=20):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=400, quote='---', n=n)
    prompt = '''I want to break down the task of figuring out whether a sentence contains anachronisms or not, into individual steps. An anachronism is a mistake in chronology, or a person, thing, or event that is out of its proper time. Here are examples of input-output pairs for the task I'm trying to break down:
----
Input: George Washington fought in the American Civil War.
Output: No
Input: The Mongolian horse rider used his bow to hunt the velociraptor.
Output: Yes
Input: Beats from the MPC3000 helped inspire many original blues artists.
Output: No
Input: Attila the Hun acted in the live-action remake of Mulan.
Output: Yes
Input: Kurt Cobain starred in the 1990 television show "Twin Peaks".
Output: Yes
----
Steps:
1.'''
    return gpt3(prompt)


In [7]:
def propose_instruction(n=20):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=400, quote='---', n=n)
    prompt = '''I want to figure out whether a sentence contains anachronisms or not. An anachronism is a mistake in chronology, or a person, thing, or event that is out of its proper time. Here are examples of input-output pairs for this task.
----
Input: George Washington fought in the American Civil War.
Output: No
Input: The Mongolian horse rider used his bow to hunt the velociraptor.
Output: Yes
Input: Beats from the MPC3000 helped inspire many original blues artists.
Output: No
Input: Attila the Hun acted in the live-action remake of Mulan.
Output: Yes
Input: Kurt Cobain starred in the 1990 television show "Twin Peaks".
Output: Yes
----
I can do this task by'''
    return gpt3(prompt)


In [8]:
decompositions = propose_decomposition(10)

In [9]:
instructions = propose_instruction(10)

In [10]:
for d in decompositions:
    print(d)
    print('---')

 Read the sentence.
2. Identify the people, things, and events mentioned in the sentence.
3. Research the people, things, and events to determine when they existed.
4. Compare the time periods of the people, things, and events to determine if they are in the correct order.
5. If the people, things, and events are not in the correct order, the sentence contains an anachronism.
---
 Read in the sentence.
2. tokenize the sentence into a list of words.
3. For each word in the list, check whether it is a proper noun.
4. If the word is a proper noun, look it up in a database of proper nouns and their corresponding time periods.
5. If the time period associated with the proper noun is earlier than the time period of the sentence, then the sentence contains an anachronism. Otherwise, it does not.
---
 Identify the subject of the sentence.
2. Identify the verb of the sentence.
3. Identify the object of the sentence, if there is one.
4. Research the subject to determine if they are anachronistic

In [11]:
for d in instructions:
    print(d)
    print('---')

 looking for words or phrases that are out of place in the sentence. For example, in the first sentence, "George Washington" and "American Civil War" are both out of place chronologically. In the second sentence, "Mongolian horse rider" and "velociraptor" are both out of place chronologically.
---
 creating a function that takes in a sentence as input and outputs whether it contains an anachronism or not.
---
 writing a regular expression that looks for anachronisms.
---
 looking for keywords that are associated with different time periods. For example, if a sentence mentions George Washington, that is automatically not an anachronism because he was around during the 1700s. However, if a sentence mentions a velociraptor, that is an anachronism because velociraptors went extinct around 65.5 million years ago.
---
 creating a list of anachronisms and then using a for loop to check whether each word in the sentence is in the list of anachronisms. If any word in the sentence is in the list

In [23]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def get_anachronism_fn(decomposition, batch_size=10):
    decomposition = '1.'+ decomposition
    last_n = int(re.findall(r'(\d+)\.', decomposition)[-1])
#     decomposition += '\n%s. Output YES if there is an anachronism, and NO otherwise' % (last_n + 1)
    def decomposition_fn(sentences):
        gpt3 = OpenAIModel(model="text-davinci-002",  max_length=400, quote='---', n=1)
        out = []
        for chunk in chunks(sentences, batch_size):
            prompts = ['''Figure out whether a sentence contains anachronisms or not, using the following steps
Steps:
%s
----
Sentence: %s
Is this an Anachronism? Output YES if there is an anachronism, and NO otherwise. Show me how you arrived at this answer step-wise.''' % (decomposition, x) for x in chunk]
            out.extend(gpt3(prompts))
        return out
    return decomposition_fn

In [18]:
def get_anachronism_ape_fn(instruction, batch_size=10):
#     decomposition = '1.'+ decomposition
#     last_n = int(re.findall(r'(\d+)\.', decomposition)[-1])
#     decomposition += '\n%s. Output YES if there is an anachronism, and NO otherwise' % (last_n + 1)
    instruction = instruction.strip()
    def decomposition_ape_fn(sentences):
        gpt3 = OpenAIModel(model="text-davinci-002",  max_length=400, quote='---', n=1)
        out = []
        for chunk in chunks(sentences, batch_size):
            prompts = ['''An anachronism is a mistake in chronology, or a person, thing, or event that is out of its proper time. Figure out whether a sentence contains anachronisms or not, using this instruction.
Instruction:
%s
----
Sentence: %s
Is this an Anachronism? Output YES if there is an anachronism, and NO otherwise.''' % (instruction, x) for x in chunk]
            out.extend(gpt3(prompts))
        return out
    return decomposition_ape_fn

In [16]:
idxs = np.random.choice(len(inputs), 100, replace=False)
labs = np.array([labels[i] for i in idxs])
subset = [inputs[i] for i in idxs]

In [19]:
preds = []
pps = []
accs = []
for z, instruction in enumerate(instructions):
    print('Instruction', z)
    fn = get_anachronism_ape_fn(instruction, batch_size=20)
    this_preds = fn(subset)
    pp = np.array([1 if 'yes' in x.lower() else 0 for x in this_preds])
    preds.append(this_preds)
    pps.append(pp)
    accs.append((pp==labs).mean())
    print((pp==labs).mean())

Instruction 0
0.67
Instruction 1
0.64
Instruction 2
0.6
Instruction 3
0.7
Instruction 4
0.56
Instruction 5
0.56
Instruction 6
0.63
Instruction 7
0.61
Instruction 8
0.68
Instruction 9
0.64


In [28]:
preds = []
pps = []
accs = []
all_preds = []
for z, decomposition in enumerate(decompositions):
    print('Decomposition', z)
    fn = get_anachronism_fn(decomposition, batch_size=20)
    this_preds = fn(subset)
    pp = np.array([1 if 'contains an anachronism' in x.lower() else 0 for x in this_preds])
    preds.append(this_preds)
    pps.append(pp)
    accs.append((pp==labs).mean())
    print((pp==labs).mean())
    all_preds.append(this_preds)

Decomposition 0
0.43
Decomposition 1
0.51
Decomposition 2
0.63
Decomposition 3
0.48
Decomposition 4
0.51
Decomposition 5
0.48
Decomposition 6
0.56
Decomposition 7
0.66
Decomposition 8
0.58
Decomposition 9
0.46


In [29]:
all_preds[0]

['\n\n1. Read the sentence.\n2. Identify the people, things, and events mentioned in the sentence.\n3. Research the people, things, and events to determine when they existed.\n4. Compare the time periods of the people, things, and events to determine if they are in the correct order.\n\nGeorge Washington Carver was an American scientist who lived from 1864 to 1943. Mahatama Gandhi was an Indian political leader who lived from 1869 to 1948. Soylent is a food product that was created in 2014.\n\n5. The people, things, and events are not in the correct order, so the sentence contains an anachronism.',
 "\n\n1. Read the sentence.\n2. Identify the people, things, and events mentioned in the sentence.\n3. Research the people, things, and events to determine when they existed.\n\nFilippo Pacini was an Italian physician who lived from 1788 to 1867. Cholera is a disease that was first described in the early 19th century.\n\n4. Compare the time periods of the people, things, and events to determ

In [30]:
subset[0]

'George Washington Carver told Mahatama Gandhi to eat more Soylent to improve his nutrition.'

In [ ]:
best = []
best_accs = []
for x in np.argsort(accs[-10:]):
    fn = get_anachronism_fn(decompositions[x], batch_size=20)
    this_preds = fn(inputs)
    best.append(this_preds)
    pp = np.array([1 if x.lower().endswith('yes') else 0 for x in this_preds])
    acc = (pp==labels).mean()
    best_accs.append(acc)
    print(acc)

In [258]:
[accs[i] for i in np.argsort(accs)[-10:]]

[0.7, 0.7, 0.7, 0.7, 0.72, 0.72, 0.72, 0.72, 0.74, 0.76]

In [228]:
accs[10]

0.85

In [221]:
nidxs = np.random.choice(len(inputs), 50, replace=False)
nlabs = np.array([labels[i] for i in nidxs])
nsubset = [inputs[i] for i in nidxs]
this_preds = fn(subset)
pp = np.array([1 if x.lower().endswith('yes') else 0 for x in this_preds])
print((pp==labs).mean())

0.7


In [229]:
fn = get_anachronism_fn(decompositions[4], batch_size=20)
this_preds = fn(inputs)

In [224]:
pp = np.array([1 if x.lower().endswith('yes') else 0 for x in this_preds])
print((pp==labels).mean())

0.7347826086956522


In [230]:
pp = np.array([1 if x.lower().endswith('yes') else 0 for x in this_preds])
print((pp==labels).mean())

0.6521739130434783


In [58]:
this_preds

[' The scientist, his cell phone, the Perserverance rover\n2. The scientist and the Perserverance rover are from 2020. His cell phone is from 2010.\n3. Yes',
 ' Read the sentence.\n2. Identify the people, things, or events in the sentence.\nHagia Sophia, church, mosque, museum\n3. Determine when each person, thing, or event in the sentence took place.\nHagia Sophia- 532 AD, church- 1000s AD, mosque- 1500s AD, museum- 1934 AD\n4. Compare the times of the people, things, or events to see if they are in the correct order.\nHagia Sophia, church, mosque, museum\n5. If the people, things, or events are in the correct order, output "No". If the people, things, or events are not in the correct order, output "Yes".\nNo',
 ' Read the sentence.\n2. Identify the people, things, or events in the sentence.\n\nPeople: President George W. Bush, generals\n\nThings: Oval Office\n\nEvents: Gulf War\n\n3. Determine when each person, thing, or event in the sentence took place.\n\nPeople: President George W

In [ ]:
pp = np.array([1 if x.lower().endswith('yes') else 0 for x in this_preds])

In [169]:
an = get_anachronism_fn(decompositions[0])

In [171]:
an(inputs[1])

[' The subject of the sentence is Ravi Shankar.\n2. Ravi Shankar is from the 20th century.\n3. The sentence is set in the 21st century.\n4. The time periods from steps 2 and 3 are different.\n5. The sentence contains an anachronism.\n6. YES']

In [153]:
import re

In [154]:
re.findall(r'(\d+)\.', decompositions[0], )

['2', '3', '4', '3', '5']

In [152]:
decompositions[19]

" Identify the subject of the sentence.\n2. Research the subject to determine its time period.\n3. Compare the subject's time period to the time period mentioned in the sentence.\n4. If the subject's time period is different from the time period mentioned in the sentence, the sentence contains an anachronism."

In [109]:
def anachronism(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=400, quote='---', n=1)
    prompt = '''Figure out whether a sentence contains anachronisms or not, using the following steps
Steps:
1. Read the sentence and identify the subject(s) and verb(s).
2. Research the subject(s) and verb(s) to see if they are appropriate for the time period mentioned in the sentence.
3. If the subject(s) and verb(s) are appropriate for the time period, then the sentence does not contain an anachronism. If the subject(s) and verb(s) are not appropriate for the time period, then the sentence does contain an anachronism.
4. Output YES if there is an anachronism, and NO otherwise
----
Sentence: %s
1.''' % x
    return gpt3(prompt)

In [112]:
idxs = np.random.choice(len(inputs), 20, replace=False)

In [130]:
preds = []
for x in inputs:
    a = anachronism(x)
    preds.append(a)

In [131]:
labs = np.array([labels[i] for i in idxs])
pp = np.array([1 if x[0].lower().endswith('YES') else 0 for x in preds])
pp2 = [0 if x[0].endswith('NO') else 1 for x in preds]

In [134]:
(pp == np.array(labels)).mean()

0.7086956521739131

In [105]:
inputs[0]

'Dwight Eisenhower was a Chief Master Sergeant of the Space Force.'

In [108]:
inputs[1]

"Ravi Shankar's two favorite musicians were Bessie Smith and Shakira."

In [110]:
a = anachronism(inputs[1])
print(a[0])

 Read the sentence and identify the subject(s) and verb(s).
Subject(s): Ravi Shankar, Bessie Smith, Shakira
Verb(s): were

2. Research the subject(s) and verb(s) to see if they are appropriate for the time period mentioned in the sentence.
Ravi Shankar was an Indian musician born in 1920. Bessie Smith was an American singer born in 1894. Shakira is a Colombian singer born in 1977.

3. If the subject(s) and verb(s) are appropriate for the time period, then the sentence does not contain an anachronism. If the subject(s) and verb(s) are not appropriate for the time period, then the sentence does contain an anachronism.
The sentence does not contain an anachronism.

4. Output YES if there is an anachronism, and NO otherwise
NO


In [32]:
def anachronism(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence and the time periods of each entity in it, tell me if it could have happened or not.
Sentence: I wrote about shakespeare
Entities and dates:
I -> 21st century
Shakespeare -> 16th century
Could the sentence be true based on the dates alone: Yes
----
Sentence: Shakespeare wrote about me

Entities and dates:
Shakespeare -> 16th century
I -> 21st century

Could the sentence be true based on the dates alone: No
----
Sentence: %s''' % x
    return gpt3(prompt)


In [69]:
def anachronismv2(x):
    gpt3 = OpenAIModel(model="text-davinci-002",  max_length=200, quote='---', n=1)
    prompt = '''Given a sentence tell me if it could have happened or not based on the time periods of the entities.
Sentence: I wrote about shakespeare
Could the sentence be true based on the time periods alone: Yes
----
Sentence: Shakespeare wrote about me
Could the sentence be true based on the time periods alone: No
----
Sentence: %s''' % x
    return gpt3(prompt)

In [78]:
def anachronismv3(x):
    prompt = '''Given a sentence, extract the entities and their relationships, and tell me if they are anachronistic.
Sentence: I wrote about shakespeare
Relationships:
- (I, write about, shakespeare) -> not anachronistic, 
----
Sentence: Shakespeare wrote about me
Relationships:
- (shakespeare, write about, me) -> anachronistic
----
Sentence: The builders of the pyramids at Giza listened to jazz during their break.
Relationships:
- (builders, listen to, jazz) -> anachronistic
----
Sentence: %s
''' % x
    return gpt3(prompt)

In [41]:
answers = []
for x in inputs:
    answers.append(anachronism(x))

In [70]:
answers2 = []
for x in inputs:
    answers2.append(anachronismv2(x))

In [79]:

answers3 = []
for x in inputs:
    answers3.append(anachronismv3(x))

In [56]:
preds = np.array([int(x[0].endswith('No')) for x in answers])

In [71]:
preds2 = np.array([int(x[0].endswith('No')) for x in answers2])

In [83]:
preds3 = np.array([1 - int('not anachronistic' in x[0]) for x in answers3])

In [68]:
(preds == labels).mean()

0.7391304347826086

In [72]:
(preds2 == labels).mean()

0.6956521739130435

In [85]:
(preds3 == labels).mean()

0.6304347826086957

In [67]:
for x in np.where(preds != labels)[0]:
    print(inputs[x])
    print(answers[x][0])
    print(labels[x])
    print('----')

Jason, the first person to explore the ancient pyramid in over 2000 years, opened a sealed box inside and found a floppy disk.


Jason -> 21st century
ancient pyramid -> ?

Could the sentence be true based on the dates alone:

The sentence could be true if the ancient pyramid is from the 21st century.
1
----
The first ever movie in color depicted the life of Sacagawea.


Entities and dates:
The first ever movie in color -> late 19th century
Sacagawea -> late 18th century - early 19th century

Could the sentence be true based on the dates alone: No
0
----
Jason, while exploring an ancient pyramid, opened a sealed box inside and found a floppy disk.


Entities and dates:
Jason -> 21st century
floppy disk -> 1970s-2000s

Could the sentence be true based on the dates alone: No
0
----
During their meetings in Bali, George Washington and the delegate of the Tokugawa shogunate exchanged gifts.


Entities and dates:
George Washington -> 18th century
Tokugawa shogunate -> 17th century

Could th